In [ ]:
import cv2
import numpy as np

def klasifikasi_hue(hue_value):
    # Deteksi warna berdasarkan nilai hue
    if hue_value < 61 or hue_value > 120:
        return "Matang", (250, 250, 0)  # Kuning cerah
    elif 121 <= hue_value <= 180:
        return "Mentah", (124, 252, 0)  # Hijau terang
    elif 45 < hue_value <= 75:
        return "Busuk", (150, 75, 0)  # Coklat / Merah gelap
    else:
        return "Tidak Terklasifikasi", (128, 128, 128)  # Abu

# Load gambar dari file yang kamu unggah
image_path = "path_to_your_image.png"  # Ubah dengan path yang sesuai
image = cv2.imread(image_path)
image = cv2.resize(image, (600, 400))  # Resize biar lebih ringan diproses
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

height, width = hsv.shape[:2]
grid_size = 20

output = image.copy()

# Loop per blok kecil
for y in range(0, height, grid_size):
    for x in range(0, width, grid_size):
        block = hsv[y:y+grid_size, x:x+grid_size]
        hue = block[:, :, 0]
        mean_hue = np.mean(hue)

        kategori, warna = klasifikasi_hue(mean_hue)
        cv2.rectangle(output, (x, y), (x+grid_size, y+grid_size), warna, -1)

# Gabungkan dengan overlay warna
combined = cv2.addWeighted(image, 0.6, output, 0.4, 0)

# Tampilkan hasil
cv2.imshow("Klasifikasi Kematangan Pisang", combined)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from tkinter import Tk, Label, Button, filedialog, StringVar, OptionMenu, Frame
from PIL import Image, ImageTk

# Klasifikasi berdasarkan hue
def klasifikasi_hue(hue_value):
    if hue_value < 30:
        return "Matang"
    elif hue_value < 50:
        return "Mentah"
    else:
        return "Busuk"

# Warna lingkaran
def warna_bulat(kategori):
    if kategori == "Mentah":
        return (0, 255, 0)
    elif kategori == "Matang":
        return (0, 255, 255)
    else:
        return (19, 69, 139)

# Pilih gambar
def pilih_gambar():
    global original_image
    path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if path:
        image = cv2.imread(path)
        image = cv2.resize(image, (300, 225))
        original_image = image.copy()
        tampilkan_gambar(image, panel_asli)
        label_status.config(text="Gambar berhasil dimuat.")

# Tampilkan gambar di panel
def tampilkan_gambar(image, panel):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(image_rgb)
    img_tk = ImageTk.PhotoImage(img_pil)
    panel.config(image=img_tk)
    panel.image = img_tk

# Filtering proses
def filtering():
    global filtered_image
    if original_image is None:
        label_status.config(text="Silakan pilih gambar terlebih dahulu.")
        return

    kategori_pilihan = selected_filter.get()
    if kategori_pilihan == "":
        label_status.config(text="Silakan pilih jenis kematangan terlebih dahulu.")
        return

    hsv = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)
    h, w = hsv.shape[:2]
    grid = 20
    filtered_image = original_image.copy()

    for y in range(0, h, grid):
        for x in range(0, w, grid):
            blok = hsv[y:y+grid, x:x+grid]
            hue_mean = np.mean(blok[:, :, 0])
            klasifikasi = klasifikasi_hue(hue_mean)

            if klasifikasi == kategori_pilihan:
                warna = warna_bulat(klasifikasi)
                center = (x + grid // 2, y + grid // 2)
                cv2.circle(filtered_image, center, 5, warna, -1)

    tampilkan_gambar(filtered_image, panel_filter)
    label_status.config(text=f"Hasil filtering: {kategori_pilihan}")

# ------------------- GUI SETUP ------------------- #
root = Tk()
root.title("Deteksi Kematangan Pisang")
root.geometry("800x400")

# Inisialisasi variabel
selected_filter = StringVar()
selected_filter.set("")
original_image = None
filtered_image = None

# Label status
label_status = Label(root, text="Silakan pilih gambar", font=("Arial", 12))
label_status.pack(pady=5)

# Frame Utama
frame_utama = Frame(root)
frame_utama.pack()

# --------- KIRI: Gambar Asli ---------
frame_kiri = Frame(frame_utama)
frame_kiri.grid(row=0, column=0, padx=10)

panel_asli = Label(frame_kiri)
panel_asli.pack()

btn_pilih = Button(frame_kiri, text="Pilih Gambar", command=pilih_gambar)
btn_pilih.pack(pady=5)

# --------- TENGAH: Panah & Dropdown ---------
frame_tengah = Frame(frame_utama)
frame_tengah.grid(row=0, column=1, padx=10)

label_panah = Label(frame_tengah, text="➡", font=("Arial", 30))
label_panah.pack(pady=10)

opsi = OptionMenu(frame_tengah, selected_filter, "Mentah", "Matang", "Busuk")
opsi.config(width=10)
opsi.pack()

# --------- KANAN: Gambar Filter ---------
frame_kanan = Frame(frame_utama)
frame_kanan.grid(row=0, column=2, padx=10)

panel_filter = Label(frame_kanan)
panel_filter.pack()

btn_filter = Button(frame_kanan, text="Mulai Filtering", command=filtering)
btn_filter.pack(pady=5)

# ------------------- MAINLOOP ------------------- #
root.mainloop()